In [70]:
import pandas as pd
import seaborn as sns
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from IPython.display import display
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
alcohol = "indicator alcohol consumption  20100830.xlsx"
gdp = "GDPpercapitaconstant2000US.xlsx"

data_alco = pd.read_excel(alcohol, header=0)
data_gdp = pd.read_excel(gdp, header=0)
data_gdp = data_gdp.set_index('Income per person (fixed 2000 US$)')
data_gdp.index.name = 'country'
data_alco.index.name = 'country'
countries = ['Portugal', 'Spain', 'United States', 'United Kingdom', 'Finland', 'Germany', 'Russia']
data_alco, data_gdp = data_alco.iloc[:, -10:], data_gdp.iloc[:, -10:]
data_alco, data_gdp = data_alco[data_alco.index.isin(countries)], data_gdp[data_gdp.index.isin(countries)]
data_gdp = data_gdp.loc[:, ('2005', '2008')]
data_alco = data_alco.loc[:, (2005, 2008)]
data_alco = data_alco.unstack().reset_index().rename({'level_0': 'year', 0: 'alco'}, axis=1)
data_gdp = data_gdp.unstack().reset_index().rename({'level_0': 'year', 0: 'gdp'}, axis=1)
data_alco['year'] = [str(i) for i in data_alco['year']]

In [3]:
data = pd.merge(data_alco, data_gdp, how='inner', on=('year', 'country'))

In [4]:
data

,year,country,alco,gdp
0,2005,Finland,12.52,26435.023438
1,2005,Germany,12.81,23564.385168
2,2005,Portugal,14.55,11586.962543
3,2005,Russia,15.76,2442.962966
4,2005,Spain,11.62,15700.527610
5,2005,United Kingdom,13.37,28354.039583
6,2005,United States,9.44,37718.005367
7,2008,Finland,13.10,28789.543853
8,2008,Germany,12.14,25620.075392
9,2008,Portugal,13.89,11949.114644


In [5]:
country = widgets.Dropdown(options=['All'] + countries, value='All', description='Country: ')
year = widgets.RadioButtons(options=list(data.year.unique()), value='2008', description='Year: ')

In [42]:
year

RadioButtons(description='Year: ', index=1, options=('2005', '2008'), value='2008')

In [45]:
def plotting(year):
    fig = plt.figure(figsize=(15, 10))
    sns.scatterplot(data['alco'][data['year']==year], 
                data['gdp'][data['year']==year], hue='country', 
                data=data)
    plt.title(f'Alcohol consumption & gdp in {year} year')
    plt.show(block=True)


In [46]:
interactive(plotting, year=year)

interactive(children=(RadioButtons(description='Year: ', index=1, options=('2005', '2008'), value='2008'), Out…

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [66]:
output_notebook()
def plotiing_bokeh(year):
    p = figure(plot_width=600, plot_height=600)
    p.circle(data['alco'][data['year']==year], data['gdp'][data['year']==year], size=15)
    show(p)

Loading BokehJS ...

In [67]:
w = interactive(plotiing_bokeh, year=year)

In [69]:
display(w)

interactive(children=(RadioButtons(description='Year: ', options=('2005', '2008'), value='2005'), Output(outpu…

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [88]:
data['alco'][data['year']=='2008'].tolist()

[13.1, 12.14, 13.89, 16.23, 11.83, 13.24, 9.7]

In [96]:
trace0 = go.Scatter(
    x=data['alco'][data['year']=='2008'].tolist(),
    y=data['gdp'][data['year']=='2008'].tolist(),
    text=list(data['country'][data['year']=='2008']),
    mode='markers',
    name='2008'
)
trace1 = go.Scatter(
    x=data['alco'][data['year']=='2005'].tolist(),
    y=data['gdp'][data['year']=='2005'].tolist(),
    text=list(data['country'][data['year']=='2005']),
    mode='markers',
    name='2005'
)
dt = [trace0, trace1]
updatemenus = list([
    dict(
        x = 0.1,
        xanchor = 'left',
        y = 1.0,
        yanchor = 'top'),
    dict(buttons=list(
        [
            dict(args = ["visible", [True, False]],
                 label = '2008',
                 method = 'restyle'),
            dict(args = ["visible", [False, True]],
                 label = '2005',
                 method = 'restyle'),
            dict(args = ["visible", [True, True]],
                 label = 'All',
                 method = 'restyle')
        ]),
    )
])


layout = dict(
            title = 'Alcohol consumption & gdp', 
            updatemenus=updatemenus, margin = dict( t=30, b=30, l=0, r=0 )
            )

fig = dict(data=dt, layout=layout)
iplot(fig)
